In [151]:
import numpy as np
import pandas as pd
import pandas as pd
import re

### Extract courses data for reco model


In [259]:
coursedata = '/Users/somethingcreative-7/Documents/Tessas Internship/code/coursefeaturesoutput.xlsx'
coursedata = pd.read_excel(coursedata)

In [260]:
new_df_courses=coursedata
new_df_courses.head()

,course_name,tags
0,Write A Feature Length Screenplay For Film Or ...,write a feature length screenplay for film or ...
1,Business Strategy Business Model Canvas Analys...,business strategy business model canvas analys...
2,Silicon Thin Film Solar Cells,silicon thin film solar cells advanced present...
3,Finance for Managers,finance for managers intermediate number eye f...
4,Retrieve Data using Single-Table SQL Queries,retrieve data using single-table sql queries b...


### Extract target job from database (currently pulled from SkillsFuture competency framework)

In [217]:
###load job description data
userdata = '/Users/somethingcreative-7/Documents/Tessas Internship/code/JDoutput.xlsx'
userdata = pd.read_excel(userdata)
userdata['Job Role'] = userdata['Job Role'].apply(lambda x: x.lstrip(' '))
#userdata.head(5)

,Job Role,Track,Sub-track,Occupation,Skills
0,Software Engineer,Software and Applications,Software Engineering,Software Engineer,"Applications Development-Level4,Computational ..."
1,Software Architect,Software and Applications,Software Engineering,Enterprise Architect,"Applications Development-Level5,Communication ..."
2,Artificial Intelligence Applied Researcher,Data and Artificial Intelligence,Artificial Intelligence Applied Research,Artificial Intelligence Applied Researcher,"Business Innovation-Level5,Leadership Advanced..."
3,Data Engineer,Data and Artificial Intelligence,NaN,Data Engineer,"Business Needs Analysis-Level2,Leadership Inte..."
4,Chief Information Officer,Infrastructure/Software and Applications/Oper...,NaN,Chief Information Officer/Chief Technology Of...,"Budgeting-Level6,Leadership Advanced, Business..."


In [218]:
###user select from dropdown list
target_job= "Data Engineer"

In [219]:
target_job_row = userdata.loc[userdata['Job Role']==target_job]
target_job_df = pd.DataFrame(target_job_row)
target_job_df

,Job Role,Track,Sub-track,Occupation,Skills
3,Data Engineer,Data and Artificial Intelligence,NaN,Data Engineer,"Business Needs Analysis-Level2,Leadership Inte..."


In [220]:
##extract skills col required for job # start with technical skills?
skills_df=pd.DataFrame(target_job_df['Skills'])
##transpose and clean, only technical skills pulled out- contains("-Level[0-9]")
skills_df=skills_df.assign(Skills=skills_df['Skills'].str.split(",")).explode('Skills')
skills_df['Skills'] = skills_df['Skills'].apply(lambda x: x.lstrip(' '))
skills_df = skills_df[skills_df['Skills'].str.contains("-Level[0-9]")]
skills_df['skills_original']=skills_df['Skills']

In [221]:
##extract skill levels into new column
skills_df['expected_level']= skills_df['Skills']
skills_df['expected_level']= skills_df['expected_level'].apply(lambda i: i.rsplit('-Level', 1)[-1])
skills_df['Skills']= skills_df['Skills'].apply(lambda i: i.split("-Level",-1)[0])

skills_df

,Skills,skills_original,expected_level
3,Business Needs Analysis,Business Needs Analysis-Level2,2
3,Change Management,Change Management-Level3,3
3,Computational Modelling,Computational Modelling-Level3,3
3,Configuration Tracking,Configuration Tracking-Level1,1
3,Configuration Tracking,Configuration Tracking-Level1,1
3,Data Design,Data Design-Level3,3
3,Data Engineering,Data Engineering-Level3,3
3,Data Ethics,Data Ethics-Level4,4
3,Data Governance,Data Governance-Level4,4
3,Data Migration,Data Migration-Level3,3


In [222]:
#prep df for users' inputs
skills_df['user_level']=skills_df['expected_level']
skills_df=skills_df.reset_index(level=None, drop=True, inplace=False)
skills_df

,Skills,skills_original,expected_level,user_level
0,Business Needs Analysis,Business Needs Analysis-Level2,2,2
1,Change Management,Change Management-Level3,3,3
2,Computational Modelling,Computational Modelling-Level3,3,3
3,Configuration Tracking,Configuration Tracking-Level1,1,1
4,Configuration Tracking,Configuration Tracking-Level1,1,1
5,Data Design,Data Design-Level3,3,3
6,Data Engineering,Data Engineering-Level3,3,3
7,Data Ethics,Data Ethics-Level4,4,4
8,Data Governance,Data Governance-Level4,4,4
9,Data Migration,Data Migration-Level3,3,3


## user input their own actual skill level

In [223]:
print(str(len(skills_df)) + ' skills found for '+ target_job)

18 skills found for Data Engineer


In [224]:
##iterate down skills list to get user input for each skill
##user's inputs should be based on competency framwork

for i in skills_df['Skills']:
    index= skills_df.index[skills_df['Skills'] == i]
    #get user input
    text= "Enter your proficiency for " + str(i) + " (number from 0-5):"
    new = int(input(text))
    skills_df.at[index, 'user_level']= new
#final df
skills_df

Enter your proficiency for Business Needs Analysis (number from 0-5):0
Enter your proficiency for Change Management (number from 0-5):5
Enter your proficiency for Computational Modelling (number from 0-5):3
Enter your proficiency for Configuration Tracking (number from 0-5):0
Enter your proficiency for Configuration Tracking (number from 0-5):5
Enter your proficiency for Data Design (number from 0-5):3
Enter your proficiency for Data Engineering (number from 0-5):0
Enter your proficiency for Data Ethics (number from 0-5):5
Enter your proficiency for Data Governance (number from 0-5):3
Enter your proficiency for Data Migration (number from 0-5):3
Enter your proficiency for Database Administration (number from 0-5):3
Enter your proficiency for Emerging Technology Synthesis (number from 0-5):3
Enter your proficiency for Project Management (number from 0-5):3
Enter your proficiency for Quality Standards (number from 0-5):3
Enter your proficiency for Security Architecture (number from 0-5):

,Skills,skills_original,expected_level,user_level
0,Business Needs Analysis,Business Needs Analysis-Level2,2,0
1,Change Management,Change Management-Level3,3,5
2,Computational Modelling,Computational Modelling-Level3,3,3
3,Configuration Tracking,Configuration Tracking-Level1,1,5
4,Configuration Tracking,Configuration Tracking-Level1,1,5
5,Data Design,Data Design-Level3,3,3
6,Data Engineering,Data Engineering-Level3,3,0
7,Data Ethics,Data Ethics-Level4,4,5
8,Data Governance,Data Governance-Level4,4,3
9,Data Migration,Data Migration-Level3,3,3


## Returning user skill gaps 

In [225]:
## Store gap calculation as new column in df
skills_df['gap'] = skills_df.apply(lambda row: int(row.user_level) - int(row.expected_level), axis=1)    
skills_df

,Skills,skills_original,expected_level,user_level,gap
0,Business Needs Analysis,Business Needs Analysis-Level2,2,0,-2
1,Change Management,Change Management-Level3,3,5,2
2,Computational Modelling,Computational Modelling-Level3,3,3,0
3,Configuration Tracking,Configuration Tracking-Level1,1,5,4
4,Configuration Tracking,Configuration Tracking-Level1,1,5,4
5,Data Design,Data Design-Level3,3,3,0
6,Data Engineering,Data Engineering-Level3,3,0,-3
7,Data Ethics,Data Ethics-Level4,4,5,1
8,Data Governance,Data Governance-Level4,4,3,-1
9,Data Migration,Data Migration-Level3,3,3,0


In [226]:
###differentiate between skills that the user already has and those that are lacking
##pull out gaps only, into new df
gaps_df = skills_df.loc[skills_df['gap'] < 0]
##reset index for gaps_df
gaps_df=gaps_df.reset_index(level=None, drop=True, inplace=False)

## Return list of skill gaps (list as input for Reco/similarity measure)
gaps_df

,Skills,skills_original,expected_level,user_level,gap
0,Business Needs Analysis,Business Needs Analysis-Level2,2,0,-2
6,Data Engineering,Data Engineering-Level3,3,0,-3
8,Data Governance,Data Governance-Level4,4,3,-1
11,Emerging Technology Synthesis,Emerging Technology Synthesis-Level4,4,3,-1
12,Project Management,Project Management-Level4,4,3,-1
13,Quality Standards,Quality Standards-Level4,4,3,-1
17,System Integration,System Integration-Level4,4,3,-1


#end